In [38]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from sklearn.metrics import mean_absolute_error

In [39]:
dati = pd.read_csv('bitcoin_data_28-4-2013_4-3-2024_granularità_1_giorno.csv')
dati

,Timestamp,Price (€)
0,2013-04-28 00:00:00.000,103.186200
1,2013-04-29 00:00:00.000,110.374500
2,2013-04-30 00:00:00.000,106.312800
3,2013-05-01 00:00:00.000,88.629100
4,2013-05-02 00:00:00.000,80.592300
...,...,...
3957,2024-02-29 00:00:00.000,57717.736389
3958,2024-03-01 00:00:00.000,56734.564616
3959,2024-03-02 00:00:00.000,57560.483900
3960,2024-03-03 00:00:00.000,57229.676585


### Curve_fit

In [46]:
# Contatore globale
eval_count = 0

# Legge da fittare: y = A * x^B
def legge(x, A, B, C, D, E, F):
    global eval_count
    eval_count += 1
    return A * x**B + C * np.sin(D*x + E) + F

# Creiamo un array per l'asse x (tempo o indice dei prezzi)
x = np.arange(1, len(dati) + 1)
y = dati['Price (€)'].values

# Ottimizzazione dei parametri
try:
    # curve_fit cerca i migliori parametri A e B
    popt, pcov = curve_fit(legge, x, y, p0=[1, 1, 1, 1, 1, 0], maxfev=10000)

    # Generazione dei valori predetti
    y_pred = legge(x, *popt)

    # Calcolo degli errori
    mae = mean_absolute_error(y, y_pred)
    mape = np.mean(np.abs((y - y_pred) / y)) * 100

    print(f"Parametri ottimizzati: {popt}")
    print(f"MAE: {mae}")
    print(f"MAPE: {mape}%")
    print(f"Numero di valutazioni della funzione: {eval_count}")

except RuntimeError as e:
    print("Errore nell'ottimizzazione:", e)

Parametri ottimizzati: [ 4.54751661e-03  1.92959282e+00  2.94437752e+01  9.97093830e-01
 -1.27923201e+01 -1.78820149e+03]
MAE: 5359.841786647315
MAPE: 234.73707941722188%
Numero di valutazioni della funzione: 324


In [47]:
pcov

array([[ 3.85425086e-06, -1.02818258e-04,  1.65466477e-04,
        -1.07061874e-08,  9.25224685e-06, -4.44139248e-01],
       [-1.02818258e-04,  2.74454381e-03, -4.28154896e-03,
         2.97210657e-07, -2.64182649e-04,  1.16640543e+01],
       [ 1.65466477e-04, -4.28154896e-03,  3.29326561e+04,
         5.83516071e-06, -3.02750114e-01, -3.99194273e+01],
       [-1.07061874e-08,  2.97210657e-07,  5.83516071e-06,
         2.90948007e-05, -5.76529157e-02,  2.80914444e-04],
       [ 9.25224685e-06, -2.64182649e-04, -3.02750114e-01,
        -5.76529157e-02,  1.52290467e+02,  5.93492996e-01],
       [-4.44139248e-01,  1.16640543e+01, -3.99194273e+01,
         2.80914444e-04,  5.93492996e-01,  8.75354767e+04]])

### Minimize

In [57]:
from scipy.optimize import minimize
import numpy as np
from sklearn.metrics import mean_absolute_error

# Funzione di errore con regolarizzazione
def loss(params, x, y, lambda_reg):
    A, B, C, D, E, F = params
    y_pred = A * x**B + C * np.sin(D * x + E) + F
    mse = np.mean((y - y_pred)**2)
    reg = lambda_reg * (F**2)  # Penalizzazione
    return mse + reg

# Ottimizzazione
initial_params = [1, 1, 1, 1, 1, 1]
result = minimize(loss, initial_params, args=(x, y, 0.01))  # lambda_reg=0.01

# Parametri ottimizzati
optimized_params = result.x
A, B, C, D, E, F = optimized_params

# Calcolo dei valori predetti
y_pred = A * x**B + C * np.sin(D * x + E) + F

# Calcolo MAE
mae = mean_absolute_error(y, y_pred)

# Calcolo MAPE
mape = np.mean(np.abs((y - y_pred) / y)) * 100

# Numero di valutazioni della funzione
n_evaluations = result.nfev

print(f"MAE: {mae}")
print(f"MAPE: {mape}%")
print(f"Numero di valutazioni della funzione: {n_evaluations}")

MAE: 5351.342789654394
MAPE: 228.69943966435477%
Numero di valutazioni della funzione: 1895


In [58]:
optimized_params

array([ 4.22160214e-03,  1.93841250e+00, -2.32314951e+01,  7.89011201e-01,
        4.36347449e+02, -1.71359330e+03])

In [59]:
result

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 65181055.29958218
        x: [ 4.222e-03  1.938e+00 -2.323e+01  7.890e-01  4.363e+02
            -1.714e+03]
      nit: 169
      jac: [-2.059e+05 -1.098e+04  5.000e-01 -4.233e+04 -7.000e+00
            -1.500e+00]
 hess_inv: [[ 3.658e-11 -1.072e-09 ...  5.480e-07  9.438e-08]
            [-1.072e-09  3.145e-08 ... -1.607e-05 -2.796e-06]
            ...
            [ 5.480e-07 -1.607e-05 ...  1.934e-02 -2.021e-03]
            [ 9.438e-08 -2.796e-06 ... -2.021e-03  5.910e-03]]
     nfev: 1895
     njev: 269

### Reti neurali

In [60]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [65]:
# Funzione per creare e compilare il modello
def create_model(input_shape, width, depth):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(input_shape,)))
    for _ in range(depth):
        model.add(tf.keras.layers.Dense(width, activation='relu'))
    model.add(tf.keras.layers.Dense(1))  # Uscita singola per la previsione del prezzo
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mse')
    return model

# Funzione per dividere i dati in train, validation e test
def split_data(x, y, train_size=0.7, val_size=0.15, test_size=0.15):
    x_train, x_temp, y_train, y_temp = train_test_split(x, y, train_size=train_size, random_state=42, shuffle=False)
    val_ratio = val_size / (val_size + test_size)
    x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, train_size=val_ratio, random_state=42, shuffle=False)
    return x_train, x_val, x_test, y_train, y_val, y_test

# Funzione per allenare il modello
def train_model(model, x_train, y_train, x_val, y_val, patience=10, epochs=100):
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=patience, restore_best_weights=True
    )
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=epochs,
        callbacks=[early_stopping],
        batch_size=32,
        verbose=2
    )
    return history

# Funzione per valutare il modello
def evaluate_model(model, x, y):
    y_pred = model.predict(x).flatten()
    mae = mean_absolute_error(y, y_pred)
    mape = np.mean(np.abs((y - y_pred) / y)) * 100
    return mae, mape

# Funzione principale
def main(data, train_size=0.7, val_size=0.15, test_size=0.15, width=64, depth=3, patience=10):
    # Prepara i dati
    x = np.arange(1, len(data) + 1).reshape(-1, 1)  # Indici come input
    y = data['Price (€)'].values  # Prezzi come target

    x_train, x_val, x_test, y_train, y_val, y_test = split_data(x, y, train_size, val_size, test_size)

    # Crea il modello
    model = create_model(input_shape=x_train.shape[1], width=width, depth=depth)

    # Allena il modello
    print("Training the model...")
    train_model(model, x_train, y_train, x_val, y_val, patience=patience)

    # Valuta il modello
    print("Evaluating the model...")
    train_mae, train_mape = evaluate_model(model, x_train, y_train)
    val_mae, val_mape = evaluate_model(model, x_val, y_val)
    test_mae, test_mape = evaluate_model(model, x_test, y_test)

    # Risultati
    print(f"Train MAE: {train_mae}, Train MAPE: {train_mape}%")
    print(f"Validation MAE: {val_mae}, Validation MAPE: {val_mape}%")
    print(f"Test MAE: {test_mae}, Test MAPE: {test_mape}%")

In [66]:
main(dati, train_size=0.7, val_size=0.15, test_size=0.15, width=1024, depth=5, patience=10)

c:\Users\Alessandro\Desktop\Cartelle\Mie\Programmi personali in Python\Kaggle\Bitcoin Historical Dataset\trading_env\lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Training the model...
Epoch 1/100
87/87 - 9s - 104ms/step - loss: 6576272.0000 - val_loss: 900428416.0000
Epoch 2/100
87/87 - 4s - 50ms/step - loss: 5832108.5000 - val_loss: 883362560.0000
Epoch 3/100
87/87 - 4s - 48ms/step - loss: 5981157.0000 - val_loss: 837452224.0000
Epoch 4/100
87/87 - 4s - 49ms/step - loss: 5673934.5000 - val_loss: 802561664.0000
Epoch 5/100
87/87 - 5s - 53ms/step - loss: 5412712.0000 - val_loss: 897176896.0000
Epoch 6/100
87/87 - 4s - 48ms/step - loss: 5552547.5000 - val_loss: 895198208.0000
Epoch 7/100
87/87 - 4s - 48ms/step - loss: 5539336.0000 - val_loss: 990580160.0000
Epoch 8/100
87/87 - 4s - 51ms/step - loss: 5540361.5000 - val_loss: 818008256.0000
Epoch 9/100
87/87 - 5s - 56ms/step - loss: 5479924.0000 - val_loss: 823082304.0000
Epoch 10/100
87/87 - 5s - 53ms/step - loss: 5408308.5000 - val_loss: 854647680.0000
Epoch 11/100
87/87 - 4s - 50ms/step - loss: 5437363.5000 - val_loss: 858564096.0000
Epoch 12/100
87/87 - 5s - 57ms/step - loss: 5370140.5000 - val